# Trainning

In [4]:
import os

import numpy as np
import mne
import joblib

from scipy.signal import filtfilt, butter  
from mne.decoding import CSP

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split

from openbci_stream.utils.hdf5 import HDF5Reader
from gcpds.mi import CSPEpochsFilterExtractor

In [6]:
with HDF5Reader(os.path.join('databases', 'motor_imagery-01_19_22-20_07_03.h5')) as reader:
    print(reader)
    
    eeg = reader.eeg
    aux = reader.aux
    timestamp = reader.timestamp
    aux_timestamp = reader.aux_timestamp
    header = reader.header
    markers = reader.markers
    fs = reader.header['sample_rate']

databases/motor_imagery-01_19_22-20_07_03.h5
2022-01-19 20:07:03.725021
MARKERS: ['Up', 'Right', 'Left', 'Bottom']
SAMPLE_RATE: 1000
STREAMING_SAMPLE_RATE: 1000
DATETIME: 1642640823.725021
MONTAGE: standard_1020
CHANNELS: {1: 'Fp1', 2: 'Fp2', 3: 'F7', 4: 'F3', 5: 'Fz', 6: 'F4', 7: 'F8', 8: 'T7', 9: 'C3', 10: 'Cz', 11: 'C4', 12: 'T8', 13: 'P7', 14: 'P3', 15: 'P8', 16: 'P4'}
CHANNELS_BY_BOARD: [16]
SHAPE: [16, 1145997]


In [7]:
data, classes = reader.get_data(tmax=4, tmin=-0.5, markers=['Right', 'Left', 'Bottom', 'Up'])
data.shape  # trials, channels, time

(180, 16, 4500)

In [8]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data, classes, test_size=0.3, random_state=123)
Xtrain.shape

(126, 16, 4500)

In [11]:
# arreglo con bandas de interes alpha y beta
f_frec = np.array([[1, 4], [4, 8], [8, 12], [12, 30]])
csp = CSPEpochsFilterExtractor(fs=fs, f_frec=f_frec, ncomp=data.shape[1], reg='empirical')  # construir modelo csp

# se crean listas de steps (pipeline)
steps = [('c_', csp),
         ('nor', StandardScaler()),
         ('cla', SVC(kernel='rbf')),
         ]  # Pipeline1

parameters = {'cla__C': [1, 10, 1e2, 1e3, 1e4],
              # Pipeline1 - hyperparametros
              'cla__gamma': [1e-3, 1e-2, 1e-1, 1, 10]
              }

label_models = ['StandarScaler_SVCrbf']

grid_search = GridSearchCV(Pipeline(steps), parameters, n_jobs=-1, cv=10, scoring='accuracy', refit=True, verbose=10) 
grid_search.fit(Xtrain, ytrain)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('c_',
                                        CSPEpochsFilterExtractor(f_frec=array([[ 1,  4],
       [ 4,  8],
       [ 8, 12],
       [12, 30]]),
                                                                 fs=1000,
                                                                 ncomp=16)),
                                       ('nor', StandardScaler()),
                                       ('cla', SVC())]),
             n_jobs=-1,
             param_grid={'cla__C': [1, 10, 100.0, 1000.0, 10000.0],
                         'cla__gamma': [0.001, 0.01, 0.1, 1, 10]},
             scoring='accuracy', verbose=10)

In [14]:
joblib.dump(grid_search, os.path.join('models', 'model_yn.pkl'))

['models/model_yn.pkl']

In [16]:
#test

model = joblib.load(os.path.join('models', 'model_yn.pkl'))
input_ = np.random.normal(0, 2, size=(1, 16, 4500))#.reshape(1, -1)
input_.shape

(1, 16, 4500)

In [17]:
 model.predict(input_) # 1, 2, 3, 4

array([4])

[CV 9/10; 1/25] START cla__C=1, cla__gamma=0.001................................
[CV 9/10; 1/25] END .cla__C=1, cla__gamma=0.001;, score=0.250 total time=  24.8s
[CV 4/10; 3/25] START cla__C=1, cla__gamma=0.1..................................
[CV 4/10; 3/25] END ...cla__C=1, cla__gamma=0.1;, score=0.308 total time=  23.6s
[CV 5/10; 4/25] START cla__C=1, cla__gamma=1....................................
[CV 5/10; 4/25] END .....cla__C=1, cla__gamma=1;, score=0.308 total time=  23.8s
[CV 7/10; 5/25] START cla__C=1, cla__gamma=10...................................
[CV 7/10; 5/25] END ....cla__C=1, cla__gamma=10;, score=0.250 total time=  24.3s
[CV 9/10; 6/25] START cla__C=10, cla__gamma=0.001...............................
[CV 9/10; 6/25] END cla__C=10, cla__gamma=0.001;, score=0.250 total time=  24.9s
[CV 2/10; 8/25] START cla__C=10, cla__gamma=0.1.................................
[CV 2/10; 8/25] END ..cla__C=10, cla__gamma=0.1;, score=0.308 total time=  23.0s
[CV 3/10; 9/25] START cla__C